<a href="https://colab.research.google.com/github/dcastf01/creating_adversarial_images/blob/main/extract_data_from_models_to_adversarial_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import os, sys, math
import numpy as np
from matplotlib import pyplot as plt
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.x
  
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE


Tensorflow version 2.4.1


In [2]:
!nvidia-smi

Fri Jan 29 09:53:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#download dataset from drive

In [3]:
import time
start_time = time.time()

##primera forma
pendiente:añadir los arhivos correctos

In [4]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"
   #URL=" https://docs.google.com/open?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    



In [5]:
# %%capture
class datasets_active:
  imagenet_validation=False  #@param {type:"boolean"} 
  Cesar_image_perturbedpertBig_round1=False  #@param {type:"boolean"} 
  Cesar_image_perturbedpertBig_round2=False  #@param {type:"boolean"} 
  Cesar_image_perturbedpertBig_round3=False  #@param {type:"boolean"} 
  Cesar_image_perturbedpertBig_round4=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round4=False#@param {type:"boolean"} 
  
  David_image_perturbedpertBig_round5_075=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round5_05=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round5_025=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round5_01=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round5_005=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round5_002=False#@param {type:"boolean"} 

  David_image_perturbedpertBig_round6_075=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round6_05=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round6_025=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round6_01=False#@param {type:"boolean"}
  David_image_perturbedpertBig_round6_005=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_round6_002=False#@param {type:"boolean"} 

  David_image_perturbedpertBig_mod_3_round1_002=False#@param {type:"boolean"} 
  David_image_perturbedpertBig_mod_3_round2_0025=True#@param {type:"boolean"} 



destination="pertBig.zip"
def get_dictionary_file_id_and_destination(file_id,destination=destination):
  return {
       "file_id":file_id,
       "destination":destination,   
        } 


files_id_destination={
    "imagenet_validation":get_dictionary_file_id_and_destination("1QA2HeOxusHicdbUxg54M8PG2GorB8ukG",
                                                                 "ILSVRC2012_img_val.tar"),
    "Cesar_image_perturbedpertBig_round1": get_dictionary_file_id_and_destination("1LLMgbtz1ENbZR1GFNvKIGEudMceqM4ZV"  ),
    # "Cesar_image_perturbedpertBig_round2":get_dictionary_file_id_and_destination("1lKXviEtQsXA481VLO-fPs5w723d186_D"),
    # "Cesar_image_perturbedpertBig_round3":get_dictionary_file_id_and_destination("1LLMgbtz1ENbZR1GFNvKIGEudMceqM4ZV"),
    # "Cesar_image_perturbedpertBig_round4":get_dictionary_file_id_and_destination("1-00I6LoDq9EOj8OMC7RU80XPSKF6LLbE"),
    # "David_image_perturbedpertBig_round4":get_dictionary_file_id_and_destination("12TXiuSTKVqTgTa_xmK2akLtTGbp82vuO"),
    "David_image_perturbedpertBig_round5_075":get_dictionary_file_id_and_destination("10MR8Q9BZhAiLpTtO8eqkO0dTlJMU0ry5"),
    "David_image_perturbedpertBig_round5_05":get_dictionary_file_id_and_destination("1uDyCu2igIpITT8Isy1FW9n49sM1-oeDT"),
    "David_image_perturbedpertBig_round5_025":get_dictionary_file_id_and_destination("1kog15Dmyj_d6t6omQudbsA2GU3SwFalN"),
    "David_image_perturbedpertBig_round5_01":get_dictionary_file_id_and_destination("1l0DqIr76nJI63J7yzgxvyhyTjMIZqmMy"),
    "David_image_perturbedpertBig_round5_005":get_dictionary_file_id_and_destination("1IVsB5QwGt-5GfefbrvSVGFI2cGW7yIGx"),    
    "David_image_perturbedpertBig_round5_002":get_dictionary_file_id_and_destination("1MuJfPRm14dWzDXAV_JjEyRjZ3iZOh6Zw"),
    
    "David_image_perturbedpertBig_round6_075":get_dictionary_file_id_and_destination("1__vhYkd-FpNNCIYuLSyyS2huvTOfsAZT"),
    "David_image_perturbedpertBig_round6_05":get_dictionary_file_id_and_destination("19szuYAsFPVA922UpOlypz_0PFX0H7IPa"),
    "David_image_perturbedpertBig_round6_025":get_dictionary_file_id_and_destination("1RZ-VC5hx4HV9pYrvGKQ5IRAbTbzRsiBI"),
    "David_image_perturbedpertBig_round6_01":get_dictionary_file_id_and_destination("1VxSr6rZvpDxVVAmMeV7najEeiAPrtq1Z"),
    "David_image_perturbedpertBig_round6_005":get_dictionary_file_id_and_destination("1lXkFZT7rMu2urCS3U3kwy0Oubp2yfGUc"),    
    "David_image_perturbedpertBig_round6_002":get_dictionary_file_id_and_destination("1x4-pVseLzzAusfUo7lFkA5PEsV28U38p"),
    
    "David_image_perturbedpertBig_mod_3_round1_002":get_dictionary_file_id_and_destination("1Ce8mcLqTU9ogoByCZw65FyjJxKwCVR8j"),
    "David_image_perturbedpertBig_mod_3_round2_0025":get_dictionary_file_id_and_destination("1aRXJOZZf0BOUBmQEuJtY7NkK83Y-2j7H"),

    }
    
for dataset_name, dictionary_fileid_destination in files_id_destination.items():
  is_active=getattr(datasets_active,dataset_name)
  if is_active:
    destination=dictionary_fileid_destination["destination"]
    file_id=dictionary_fileid_destination["file_id"]
    download_file_from_google_drive(file_id, destination)

    !unzip -q /content/pertBig.zip
    print("the dataset :",dataset_name, "was download and unzip")


label_id="1PKH4QWZe_VCOhu19oOhbV9z-YKrKACO7"
destination_label_id="/content/label.txt"
download_file_from_google_drive(label_id, destination_label_id)

label_real_name="1sxe3eunq5U4EwsHlLaeRmwcnaZjCEcnh"
destination_label_real_name="/content/name_real_label.txt"
download_file_from_google_drive(label_real_name, destination_label_real_name)
#confirmar que se descarga bien ya que si se ha descargado varias veces al ser tan grande luego no te deja

the dataset : David_image_perturbedpertBig_mod_3_round2_0025 was download and unzip


##upload the drive to storage the information

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

El tiempo que se necesita en setup es del


#Device detection

In [ ]:
#@title Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy for hardware
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  #strategy = tf.distribute.MirroredStrategy(gpus) # this works for 1 to multiple GPUs
  print('Running on ', len(gpus), ' GPU(s) ')
  strategy=tf.distribute.OneDeviceStrategy(gpus[0])
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')

# How many accelerators do we have ?
print("Number of accelerators: ", strategy.num_replicas_in_sync)

#Functions Globals

In [ ]:
#@title functions to create model

from tensorflow.keras.models import Model
import os
import pandas as pd

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      
      plt.imshow(image_batch[n])
      #plt.title(real_class[(CLASS_NAMES[label_batch[n]==1][0])].title())
      plt.title(real_class[np.argmax([label_batch[n]])].title())
      plt.axis('off')
def get_models_made():
  path_base="/content/drive/My Drive/AI/Ejemplos adversariales/resultados2"
  files=os.listdir(path_base)
  models_made= [ model[:-4].lower() for model in files]
  return models_made

def create_model(name_model,classes=1000):
  with strategy.scope():

    model=getattr(tf.keras.applications,name_model)(include_top=True, weights='imagenet', classes=classes) #quite input shape por motivos varios
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)
    METRICS=[ 
      
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      tf.keras.metrics.Accuracy(name="accuracy_binary"),
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc',num_thresholds=500),#cambiar el auc
      tf.keras.metrics.TopKCategoricalAccuracy( k=3, name='top_3_categorical_accuracy'),
      tf.keras.metrics.TopKCategoricalAccuracy( k=5, name='top_5_categorical_accuracy'),
      tf.keras.metrics.TopKCategoricalAccuracy( k=10, name='top_10_categorical_accuracy')
        ]
  
    # model.compile(loss='categorical_crossentropy',
    #               optimizer=opt,
    #               metrics=METRICS)
    return model

    
def train_model(model,train_data,validation_data=None,callbacks=None):

  model.fit(train_data,
    #steps_per_epoch =( train_generator.samples // BATCH_SIZE),
    validation_data = validation_data, 
    #alidation_steps = (validation_generator.samples // BATCH_SIZE),
    epochs = 100,
   #callbacks=callbacks
   )
  


#Models API Tensorflow

##functions necessary

In [ ]:
import numpy as np 
import pathlib
import time

In [ ]:
#@title creating functions to create model and extract preprocesing input
def clean_list_of_models_names(lists_of_private_object):
  #todos los modelos empiezan por mayuscula y no son privados, es decir no pueden empezar con "_"
  prev_clean_list=list()
  clean_list=list()
  for element in lists_of_private_object:
    if element[0]=="_" or not element[0].isupper():
      pass
    else:
      
      prev_clean_list.append(element)
  return prev_clean_list

def clean_list_of_familys(lists_of_private_object):
  #todos los modelos empiezan por mayuscula y no son privados, es decir no pueden empezar con "_"
  prev_clean_list=list()
  clean_list=list()
  for element in lists_of_private_object:
    if element[0]=="_" or not element[0].islower() or element=="imagenet_utils":
      pass
    else:      
      prev_clean_list.append(element)
  return prev_clean_list
  
def get_preprocesing_input(ALL_FAMILYS,model):
  def check_which_is_family_is_the_family_to_which_it_belongs(ALL_FAMILYS,model):

    for family in ALL_FAMILYS:
      family_module=getattr(tf.keras.applications,family)
      if model in dir(family_module):
        return family_module
    return "ERROR, NO SE HA ENCONTRADO EN NINGUNA FAMILIA"
  def get_function_preprocess_input(module_family_net):
    return getattr(module_family_net,"preprocess_input")
  family_module=check_which_is_family_is_the_family_to_which_it_belongs(ALL_FAMILYS,model)
  preprocess_input=(get_function_preprocess_input(family_module))
  return preprocess_input

def check_is_csv_is_created(model_name):
  path_to_check=path_result+"/"+model_name+".csv"
  if os.path.isfile(path_to_check): 
    return True
  return False

def create_model(name_model,classes=1000):
  with strategy.scope():

    model=getattr(tf.keras.applications,name_model)(include_top=True, weights='imagenet', classes=classes) #quite input shape por motivos varios
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)
    METRICS=[ 
      
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      tf.keras.metrics.Accuracy(name="accuracy_binary"),
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc',num_thresholds=500),#cambiar el auc
      tf.keras.metrics.TopKCategoricalAccuracy( k=3, name='top_3_categorical_accuracy'),
      tf.keras.metrics.TopKCategoricalAccuracy( k=5, name='top_5_categorical_accuracy'),
      tf.keras.metrics.TopKCategoricalAccuracy( k=10, name='top_10_categorical_accuracy')
        ]
  
    # model.compile(loss='categorical_crossentropy',
    #               optimizer=opt,
    #               metrics=METRICS)
    return model

In [ ]:
#@title funcionts to create list and predict
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')
      
def create_list_file_ds(path):
  Imagenet_root = pathlib.Path(path)
  list_ds = tf.data.Dataset.list_files(str(Imagenet_root/'*/*'))
  return list_ds

def create_list_of_dataset_with_preprocess(filename):
  
  parts = tf.strings.split(filename, os.sep)
  
  label = parts[-2]

  image = tf.io.read_file(filename,)
  ################ try tf.cast como tiene cesar###
  
  #probar pasandolo a pil

  #probar usando tf.io.decode
  image = tf.io.decode_jpeg(image,channels=3)
  print(image)
  # image = tf.image.decode_jpeg(image,channels=3)
  image = tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
  filename_string=parts[-1]


  try:
    image=function_to_preprocess(image)
  except:  
    print("error")
    print(filename)
    print(image.shape)
    print(filename_string)
    
  return image,label,filename_string

def predict_model(model,map_de_dataset,df,model_name):
  #testeando el metodo lista, el anterior metodo está repetido con otro nombre más abajo
  start_time=time.time()
  init_batch=time.time()
  count_batch=0
  count_image=0

  list_filename=list()
  list_code_predict=list()
  list_probability_class=list()
  list_class_name_predict=list()
  list_class_code_real=list()
  list_class_name_real=list()
  list_probability_class_real=list()
  list_model=list()
  list_id_image=list()
  list_count=list()
  for image_batch, label_batch,filename_batch in map_de_dataset.batch(nbatch).take(niteration):  
    # print("filename: ", filename.numpy())
    count_batch+=1
    prediction=model.predict(image_batch.numpy())
    for image_prediction,label,filename_like_bytes in zip(prediction,label_batch,filename_batch):
      count_image+=1

      class_code_predict=int(np.argmax(image_prediction))
      list_code_predict.append(class_code_predict)
      # print("predictionlabel: ",np.argmax(image_prediction))
      probability_class=image_prediction[class_code_predict]
      list_probability_class.append(probability_class)
      # print("probability_class: ",probability_class)
      class_name_predict=real_class[class_code_predict]
      list_class_name_predict.append(class_name_predict)
      # print("class_name_predict: ",class_name_predict)

      class_code_real=int(label.numpy().decode("UTF-8"))
      list_class_code_real.append(class_code_real)
      class_name_real=real_class[class_code_real]
      list_class_name_real.append(class_name_real)
      probability_class_real=image_prediction[class_code_real]
      list_probability_class_real.append(probability_class_real)
      # print("class_code_real: ", class_code_real)
      # print("class_name_real: ",class_name_real)
      # print("probability_class_real: ", probability_class_real)

      filename=filename_like_bytes.numpy().decode('UTF-8')
      list_filename.append(filename)
      id_image=filename[-13:-5]
      list_model.append(model_name)
      list_id_image.append(id_image)
      list_count.append(count_image)
      # print("filename: ",filename)
      # print("id_image: ",id_image)
      
     
    time_batch=(time.time() - init_batch)
    if count_batch%250==0:
      print("--- %s segundos es lo que ha tardado este batch en ser predecido---" % (time_batch)) 
      print("--- %s minutos es lo que tardaría en predecir todo el conjunto de imágenes---" % (time_batch*niteration/60))  
      print("--- %s batch analizado---" % (round(count_batch/niteration*100,2))) 
  
    init_batch=time.time()
  time_create_dataframe=time.time()
  df=pd.DataFrame(list(zip(list_filename,list_code_predict,list_probability_class,list_class_name_predict,
                           list_class_code_real,list_class_name_real,list_probability_class_real,list_model,list_id_image)),
                  columns=["filename","code_predict","probability_class","class_name_predict","class_code_real",
                           "class_name_real","probability_class_real","model","id_image"],index=[list_count])
  duration_creation_dataframe=(time.time()-time_create_dataframe)
  print("--- %s segundos es lo que ha tardado en crearse el dataframe---" % (duration_creation_dataframe)) 
  time_total=(time.time() - start_time)
  print("------ %s minutos es lo que se tardo en predecir todo el conjunto de imágenes------" % (time_total/60))  
  return df


#study perturbed image

##setup

se tienen que haber descargado los datos , luego se tiene unlog.txt con la información de las imagenes y una carpeta con las imágenes

In [ ]:
#@title get labels
import pandas as pd
import pathlib
import glob
import json
import math
import time

def get_dictionary_with_real_class():
  path_json_real_label="/content/name_real_label.txt" #el json es el bueno
  real_class_prev=json.loads(open(path_json_real_label).read())
  real_class=dict()
  for k in real_class_prev:
    real_class[int(k)]=real_class_prev[k][1] #dictionary key:number value:string
  return real_class

def change_key_per_value_in_dictionary(dictionary):
  new_dict=dict()
  for k,v in dictionary.items():
    if v in new_dict.keys():
      print(k)
    new_dict[v]=k
  return new_dict
def get_code_name_prediction(name):

  return real_class_inverted[name]

real_class=get_dictionary_with_real_class()
real_class_format_to_generator= {str(k): v for k, v in real_class.items()}
real_class_inverted=change_key_per_value_in_dictionary(real_class)

In [ ]:
#@title for when we test various  experiments
logs=[file_txt[:-4] for file_txt in glob.glob("*.txt") if file_txt.split("_")[0]=="log"]
epsilons_available=[log.split("_")[-1] for log in logs]

In [ ]:
#@title predict all images with all the models
for epsilon in epsilons_available:
  print("doing", epsilon, "experiment")
  data_dir_raw = "/content/result_adversarial"#@param {type:"string"}
  data_dir_raw=data_dir_raw+epsilon
  validation_labels_file = "/content/label.txt"#@param {type:"string"}
  data_dir="/content/perturbed_order"#@param {type:"string"}
  data_dir=data_dir+epsilon
  log_dir="/content/log_e_{}.txt".format(epsilon)
  mod1=False #@param {type:"boolean"}
  mod3=True #@param {type:"boolean"}
  if mod1:
    data = pd.read_csv(log_dir, sep=",",index_col=False, header=None,
                      names=["originalfilename","original_prediction","confidence_original","epsilon","pertubed_prediction","confidence_pertubed","name_new_file"])
    data["pertubed_code"]=data["pertubed_prediction"].apply(get_code_name_prediction)
  elif mod3:
    data = pd.read_csv(log_dir, sep=",",index_col=False, header=None,
                      names=["model_name",
                             "target",
                             "originalfilename",
                             "original_prediction",
                             "confidence_original",
                             "epsilon",
                             "all_models_had_exist_creating_adversarial",
                             "confidence_pertubed",
                             "name_new_file"])
  

  data["original_code"]=data["original_prediction"].apply(get_code_name_prediction)
  
  result_dir = pathlib.Path(data_dir_raw)
  labels=data["original_code"].unique().tolist()
  unique_labels = set(labels)


  for label in unique_labels:
      labeled_data_dir = os.path.join(data_dir, str(label))
      # Catch error if sub-directory exists
      try:
        os.makedirs(labeled_data_dir)
      except OSError as e:
        print(e)

  for index, row in data.iterrows():
    try:
      perturbed_filename=row["name_new_file"]
      original_code=row["original_code"]

      perturbed_path = os.path.join(data_dir_raw, perturbed_filename)
      new_filename = os.path.join(data_dir, str(original_code), perturbed_filename)
      os.rename(perturbed_path, new_filename)
    except OSError as e:
      print(e)

  ##Prediction

  path_result="/content/drive/MyDrive/AI/Ejemplos adversariales/mod3/round2/" #@param {type:"string"}
  path_result=path_result+"{}/CSV individuales".format("0"+str(epsilon.split(".")[-1]))

  path_width_data= data_dir
  AUTOTUNE = tf.data.experimental.AUTOTUNE

  nbatch=50
  nImages=data.shape[0]
  niteration=math.ceil(nImages/nbatch)


  all_models_in_keras_without_clean=dir(tf.keras.applications)
  ALL_MODELS=clean_list_of_models_names(all_models_in_keras_without_clean)
  ALL_MODELS=ALL_MODELS[:15] +ALL_MODELS[17:]
  ALL_FAMILYS=clean_list_of_familys(all_models_in_keras_without_clean)
  
  for model_name in ALL_MODELS:
    print(model_name)
    init_model_time=time.time()
    if check_is_csv_is_created(model_name)==False:
      preprocess_input=get_preprocesing_input(ALL_FAMILYS,model_name)
      function_to_preprocess=preprocess_input
      model=create_model(model_name)
      IMG_WIDTH=model.input.shape[1]
      IMG_HEIGHT =model.input.shape[2]
      
      list_ds=create_list_file_ds(path_width_data)
      
      labeled_ds = list_ds.map(create_list_of_dataset_with_preprocess,num_parallel_calls=AUTOTUNE)

      df=pd.DataFrame()
      df=predict_model(model,labeled_ds,df,model_name)
      
      if df.shape[0]>0:
            df.to_csv(path_result+"/"+model_name+"perturbed.csv")
      else:
          print("En el modelo "+ model_name+" hubo un error, por lo que ver que paso")
      
      del (list_ds)
      del (labeled_ds)
      
      
      tf.keras.backend.clear_session() #quizá para limpiar la ram
    print("------ %s minutos es lo que se tardo en realizar todo el modelo------" % ((time.time()-init_model_time)/60))  

  # result_Directory="/content/drive/My Drive/AI/Ejemplos adversariales/resultados_api_tensorflow/prediction_models_v2"
  result_Directory=path_result
  result_dir = pathlib.Path(result_Directory)
  # CLASS_NAMES = np.array([item.name for item in result_dir.glob('.csv') ]
  filenames=[item.name for item in result_dir.glob('*.csv') ]
  path_result_join= "/content/drive/MyDrive/AI/Ejemplos adversariales/mod3/round2/" #@param {type:"string"}
  path_result_join=path_result_join+"{}".format("0"+str(epsilon.split(".")[-1]))
  

##Prediction

##ánalisis de imágenes perturbadas


In [ ]:
# # result_Directory="/content/drive/My Drive/AI/Ejemplos adversariales/resultados_api_tensorflow/prediction_models_v2"
# result_Directory=path_result
# result_dir = pathlib.Path(result_Directory)
# # CLASS_NAMES = np.array([item.name for item in result_dir.glob('.csv') ]
# filenames=[item.name for item in result_dir.glob('*.csv') ]
# # path_result_join= "/content/drive/My Drive/AI/Ejemplos adversariales/Round 5/002" #@param {type:"string"}

In [ ]:
#@title threshold
threshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
df_global=pd.DataFrame()
path_base=result_Directory+"/"
for filename in filenames:
  df_aux=pd.read_csv(path_base+filename)
  df_aux.drop(columns="Unnamed: 0",inplace=True)
  df_aux["model"]=filename[:-4]#only this time
  df_global=pd.concat([df_global,df_aux],ignore_index=True)
condition_class_predict_is_real=df_global["code_predict"]==df_global["class_code_real"]
condition_prediction_is_more_than_threshold=df_global["probability_class_real"]>=threshold
conditions=condition_class_predict_is_real & condition_prediction_is_more_than_threshold
df_global["epsilon"]= [text[9:14] for text in df_global["filename"]] #careful here with the number of digits
df_global["filename"]= [text[15:] for text in df_global["filename"]]
df_global["model"]=[text[:-9] for text in df_global["model"]]

df_global["isright"]=np.where(conditions ,1,0)
df_global["ispertubed"]=True


Realizar un multilevel con image y epsilon

In [ ]:
df_global.to_csv(path_result_join+"/perturbed_complete_df.csv")

In [ ]:
df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright")


In [ ]:
df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright").to_csv(path_result_join+"/matriz_bool_ejemplos_adversariales_perturbadas.csv")
df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="probability_class_real").to_csv(path_result_join+"/matriz_porcentaje_ejemplos_adversariales_perturbadas.csv")

In [ ]:
df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright")

In [ ]:
print("el modelo que tiene que acertar todo Mobilenet sus resultados son un accuracy de:",df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright").MobileNet.mean())
print("el modelo que tiene que fallar todo ResNet50V2 sus resultados son un accuracy de:",df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright").ResNet50V2.mean())
print("el modelo que tiene que fallar un 50% y acertar el otro 50% todo DenseNet121 sus resultados son un accuracy de:",df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright").DenseNet121.mean())

In [ ]:
#@title threshold
threshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
df_global=pd.DataFrame()
path_base=result_Directory+"/"
for filename in filenames:
  df_aux=pd.read_csv(path_base+filename)
  df_aux.drop(columns="Unnamed: 0",inplace=True)
  df_aux["model"]=filename[:-4]#only this time
  df_global=pd.concat([df_global,df_aux],ignore_index=True)
condition_class_predict_is_real=df_global["class_code_real"]==df_global["class_code_real"]
condition_prediction_is_more_than_threshold=df_global["probability_class"]>=threshold
conditions=condition_class_predict_is_real & condition_prediction_is_more_than_threshold
df_global["epsilon"]= [text[9:14] for text in df_global["filename"]] #careful here with the number of digits
df_global["filename"]= [text[15:] for text in df_global["filename"]]
df_global["model"]=[text[:-9] for text in df_global["model"]]

df_global["isright"]=np.where(conditions ,1,0)
df_global["ispertubed"]=True

In [ ]:
df_global

In [ ]:
df_global.pivot_table(index=['filename', 'epsilon'],columns="model",values="isright").mean()
